# Temperature extraction:

In [73]:
import numpy as np
import pandas as pd
import re
import glob
import os
import dateutil

We define some functions that will be used to extract temperatures from text

In [236]:
# Allows to replace some similar words by the seme words
def replace_words(string, words, token):
    for w in words:
        string = string.replace(w, token)
    
    return string


# initialize_variables():

# We will replace all words that are similar to degre with the same word:degre
words_1 = ['degré','degrés','degre','degres','degree','degrees','degrée','degrées','°','°C','° C']
token_1 = 'degre'
# Same for the term plus
words_2 = ['plus','+']
token_2 = 'plus'
# Same for the term moins
words_3 = ['moins','-']
token_3 = 'moins'
# This change is made to allow paragraph selection
words_4 = ['Rétrospective météo','Retrospective meteo']
token_4 = 'situation generale'

# This list regroups the different localizations of temperatures
location = ['nord','sud','est','ouest','nord-est','nord-ouest','sud-est','sud-ouest']
    
# Creation of lists that will contain temperatures and dates
temperature = []
temperature_nord = []
temperature_sud = []
temperature_est = []
temperature_ouest = []
date = []
date_nord = []
date_sud = []
date_est = []
date_ouest = []

We extract the temperatures from the files we have downloaded.
We previously transformed the pdf files into text files in order to do the text processing.

In [237]:
# we create the variables using the function described below
initialize_variables()
temperature = []

for year in range(2000,2018):
    # we select all the text files corresponding to 1 year
    path = "../data/slf/{}/nb/fr/txt".format(str(year))
    
    for filename in glob.glob(os.path.join(path, '*.txt')):
        
        paragraph = []
        # Opening of a text file
        #filename = "../data/slf/2000/nb/fr/txt/19991206_nb_fr_bw.txt"
        with open(filename) as file:
            
            content = file.read()
            
            # uniformize the words using the function replace_words
            content = replace_words(content, words_1, token_1)
            content = replace_words(content, words_2, token_2)
            content = replace_words(content, words_3, token_3)
            content = replace_words(content, words_4, token_4)

            # collect paragraph in which temperature is present
            for text in content.split('\n\n\n'):
                if 'situation generale' in text.lower():
                    paragraph = text
              
        
            # check if paragraph is not empty
            if paragraph != []:
                
                #Calculate number of temperatures
                num = paragraph.count('degre')
                
                # We select a string containing the temperature values
                pattern_moins = re.compile(r"moins \d+ degre", re.IGNORECASE)
                pattern_plus = re.compile(r"plus \d+ degre", re.IGNORECASE)
                # we use the patternes obtained with regex to get thright part of the paragraph
                neg_temp = pattern_moins.findall(paragraph)
                pos_temp = pattern_plus.findall(paragraph)
                
                # in case only one temperature is present in the file
                if num==1:
                # Necessity to differentiate between positive and negative temperatures (sign)
                    
                    # Negative temperatures:
                    if (neg_temp != []):
                        # Temperature is added in a list along with the date
                        temperature.append([-int(T) for T in neg_temp[0].split() if T.isdigit()][0])
                        date.append(dateutil.parser.parse(filename[27:35]))
                     
                    # Positive temperatures:
                    elif (pos_temp != []):
                        # Temperature is added in a list along with the date
                        temperature.append([int(T) for T in pos_temp[0].split() if T.isdigit()][0])
                        date.append(dateutil.parser.parse(filename[27:35]))
                
                # in case several temperatures are present in the file
                elif num>1:

                    if (pos_temp != []) | (neg_temp != []):
                        # obtain the line containing the location
                        targets = [line for line in paragraph.split('.') if 'degre' in line]
                        
                        # Create a vector containing sign, temperature and location for both temperatures  
                        vect = []
                        for i,l in enumerate(targets[0].replace("'", " ").replace(","," ").replace("\n"," ").split(' ')):
                            if len(vect)==0 and l in ['moins','plus']:
                                vect.append(l)
                            if len(vect)==1 and l.isdigit():
                                vect.append(l)
                            if len(vect)==2 and (l in location):
                                vect.append(l)
                            if len(vect)==3 and l in ['moins','plus']:
                                vect.append(l)
                            if len(vect)==4 and l.isdigit():
                                vect.append(l)
                            if len(vect)==5 and (l in location):
                                vect.append(l)
                        
                        # We update temperature vectors and dates vectors
                        if len(vect)==6:
                            for i, loc in enumerate([vect[0],vect[3]]):
                                if loc == 'plus':
                                    if pos_temp != []:

                                        if vect[2+(3*i)] == 'nord':
                                            temperature_nord.append(int(vect[1+(3*i)]))
                                            date_nord.append(dateutil.parser.parse(filename[27:35]))
                                        if vect[2+(3*i)]  == 'sud':
                                            temperature_sud.append(int(vect[1+(3*i)]))
                                            date_sud.append(dateutil.parser.parse(filename[27:35]))
                                        if vect[2+(3*i)]  == 'est':
                                            temperature_est.append(int(vect[1+(3*i)])) 
                                            date_est.append(dateutil.parser.parse(filename[27:35]))
                                        if vect[2+(3*i)]  == 'ouest':
                                            temperature_ouest.append(int(vect[1+(3*i)])) 
                                            date_ouest.append(dateutil.parser.parse(filename[27:35]))

                                if loc == 'moins':
                                    if neg_temp != []:

                                        if vect[2+(3*i)]  == 'nord':
                                            temperature_nord.append(-int(vect[1+(3*i)]))
                                            date_nord.append(dateutil.parser.parse(filename[27:35]))
                                        if vect[2+(3*i)]  == 'sud':
                                            temperature_sud.append(-int(vect[1+(3*i)])) 
                                            date_sud.append(dateutil.parser.parse(filename[27:35]))
                                        if vect[2+(3*i)]  == 'est':
                                            temperature_est.append(-int(vect[1+(3*i)])) 
                                            date_est.append(dateutil.parser.parse(filename[27:35]))
                                        if vect[2+(3*i)]  == 'ouest':
                                            temperature_ouest.append(-int(vect[1+(3*i)])) 
                                            date_ouest.append(dateutil.parser.parse(filename[27:35]))  


#print(temperature)
len(temperature) # 93/199

587

We define the dataframes

In [238]:
temp_df = pd.DataFrame({'Date':date,'Temperature':temperature})
temp_df = temp_df.set_index('Date')

temp_df_nord = pd.DataFrame({'Date':date_nord,'Temperature Nord':temperature_nord})
temp_df_nord = temp_df_nord.drop_duplicates(subset='Date', keep='first')
temp_df_nord = temp_df_nord.set_index('Date')

temp_df_sud = pd.DataFrame({'Date':date_sud,'Temperature Sud':temperature_sud})
temp_df_sud = temp_df_sud.drop_duplicates(subset='Date', keep='first')
temp_df_sud = temp_df_sud.set_index('Date')

temp_df_est = pd.DataFrame({'Date':date_est,'Temperature est':temperature_est})
temp_df_est = temp_df_est.drop_duplicates(subset='Date', keep='first')
temp_df_est = temp_df_est.set_index('Date')

temp_df_ouest = pd.DataFrame({'Date':date_ouest,'Temperature ouest':temperature_ouest})
temp_df_ouest = temp_df_ouest.drop_duplicates(subset='Date', keep='first')
temp_df_ouest = temp_df_ouest.set_index('Date')
len(temp_df_ouest.index.unique())

190

We concatenate all dataframes to get only one dataframe containing all temperatures with dates as the index

In [239]:
new_df = pd.concat([temp_df, temp_df_nord, temp_df_sud, temp_df_est, temp_df_ouest], axis=1)
len(new_df) #1292 
new_df

,Temperature,Temperature Nord,Temperature Sud,Temperature est,Temperature ouest
Date,,,,,
1999-11-22,-10.0,NaN,NaN,NaN,NaN
1999-11-23,-6.0,NaN,NaN,NaN,NaN
1999-11-24,-4.0,NaN,NaN,NaN,NaN
1999-11-26,5.0,NaN,NaN,NaN,NaN
1999-11-28,4.0,NaN,NaN,NaN,NaN
1999-11-29,6.0,NaN,NaN,NaN,NaN
1999-12-01,3.0,NaN,NaN,NaN,NaN
1999-12-03,NaN,NaN,NaN,3.0,6.0
1999-12-06,NaN,NaN,NaN,-6.0,-3.0


In [179]:
if len(new_df) == len(new_df.index.unique()):
    print("unique index")

unique index
